# Projet Traitement Large Échelle

Pour ce projet j'ai choisi d'utiliser RQLite comme base de données relationelle, et KeyDB comme base de données NoSQL.

Nous allons tout d'abord étudier KeyDB, en détaillant son fonctionnement, puis en effectuant des tests de performance.

Ensuite, nous ferons de même pour RQLite, et finalement nous comparerons les deux bases de données.

## Présentation de KeyDB

KeyDB est une version améliorée de Redis axée sur le multithreading, l'efficacité mémoire et le débit élevé.
Il conserve une compatibilité totale avec le protocole, les modules et les scripts Redis, tout en offrant des améliorations de performance telles que la réplication active et le stockage FLASH.

Grâce à son architecture MVCC, KeyDB permet l'exécution de requêtes telles que KEYS et SCAN sans bloquer la base de données ni dégrader les performances.
En utilisant le même matériel, KeyDB peut atteindre un débit significativement plus élevé que Redis.
 
Son architecture multithread simplifie la répartition des charges et permet une utilisation plus efficace des ressources matérielles. 
En outre, KeyDB offre une compatibilité avec les derniers développements de Redis, ce qui en fait un substitut direct pour les déploiements existants.

### Installation et démarrage

Il est nécessaire d'avoir docker installé sur votre machine.

In [ ]:
!docker run -name keydb-server -d eqalpha/keydb keydb-server /etc/keydb/keydb.conf --server-threads 4 --requirepass password

## Présentation de RQLite

RQLite est une base de données relationnelle qui combine la simplicité de SQLite avec la fiabilité d'un système distribué robuste. Il est facile à déployer et à utiliser. Basé sur SQLite, il assure le stockage fiable des données tout en offrant des fonctionnalités distribuées.

Quelques avantages:

- Léger et intégrable: rqlite est une base de données légère qui peut être facilement intégrée aux applications existantes.
- Distribué et tolérant aux pannes: rqlite réplique les données sur plusieurs nœuds, ce qui permet de maintenir la disponibilité des données même en cas de défaillance d'un nœud.
- Durable: rqlite prend en charge les transactions ACID, ce qui garantit l'intégrité des données.
- Scalable: rqlite peut être facilement mise à l'échelle horizontalement en ajoutant des nœuds supplémentaires au cluster.